# Install and Import Dependencies

In [12]:
import mediapipe as mp
import cv2

In [13]:
mp_drawing = mp.solutions.drawing_utils 
mp_holistic = mp.solutions.holistic

#  Make Detections

In [14]:
cap = cv2.VideoCapture(0)
# Mediapipe Holestic model initiation for integrating face, hand and pose components
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Webcam-Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #  Face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        #  Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        #  Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        #  Body Pose
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [15]:
# Result Landmarks
results.face_landmarks.landmark[0]

x: 0.47609055042266846
y: 0.4888790249824524
z: -0.02681177668273449

# 2. Capture Landmarks & Export to CSV
<!-- <img src="https://i.imgur.com/8bForKY.png"> -->
<!-- <img src="https://i.imgur.com/AzKNp7A.png"> -->

In [16]:
import csv
import os # helps with exporting to a folder structure
import numpy as np

In [17]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark) # number of landmarks for pose (468) + number of landmarks for face (33)
num_coords

501

In [18]:
# creating Header row for CSV

landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [19]:
# landmarks

In [20]:
# Exporting Co-ordinates to CSV uding csv_writer.writerow

with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [48]:
# Exporting with a class name 

class_name = "Standing Up"

In [49]:
# Simmmilar to Part 1 but this time we export co-ordinates to CSV

cap = cv2.VideoCapture(0)
# Mediapipe Holestic model initiation for integrating face, hand and pose components
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Webcam-Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #  Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        #  Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        #  Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        #  Body Pose
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# Train Custom Model Using Scikit Learn

### 1 Read in Collected Data and Process

In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [51]:
df = pd.read_csv('coords.csv')

In [74]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Thumbs Up,0.414089,0.407565,-0.926686,0.999919,0.432624,0.358319,-0.843457,0.999821,0.444706,...,-0.001313,0.0,0.468765,0.376737,0.013527,0.0,0.472230,0.372671,0.014051,0.0
1,Thumbs Up,0.414763,0.407829,-0.905378,0.999918,0.433663,0.358908,-0.829413,0.999818,0.446065,...,-0.002077,0.0,0.469554,0.377602,0.012555,0.0,0.472963,0.374379,0.013008,0.0
2,Thumbs Up,0.414994,0.408249,-0.894446,0.999919,0.434616,0.359369,-0.819523,0.999820,0.447265,...,-0.000467,0.0,0.471017,0.378174,0.014460,0.0,0.474558,0.374432,0.015072,0.0
3,Thumbs Up,0.416811,0.409154,-0.889051,0.999920,0.436196,0.360598,-0.813763,0.999821,0.448735,...,-0.001432,0.0,0.471922,0.378373,0.014184,0.0,0.475283,0.375294,0.014699,0.0
4,Thumbs Up,0.416805,0.409854,-0.871605,0.999921,0.436278,0.361166,-0.798493,0.999822,0.448885,...,-0.002613,0.0,0.471913,0.376075,0.011964,0.0,0.475247,0.373183,0.012304,0.0


In [75]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
388,Drowsy,0.324336,0.637896,-1.007702,0.991579,0.313133,0.571170,-1.032566,0.982105,0.322348,...,-0.018147,0.0,0.303159,0.570838,-0.041461,0.0,0.302816,0.562885,-0.044441,0.0
389,Standing Up,0.483750,0.047641,-0.791061,0.999933,0.508691,0.001452,-0.762163,0.999869,0.522984,...,0.000877,0.0,0.543293,-0.020725,0.017091,0.0,0.548040,-0.028908,0.018152,0.0
390,Standing Up,0.473050,0.031469,-0.445751,0.999833,0.487423,0.001360,-0.426568,0.999568,0.496246,...,0.001607,0.0,0.505612,0.043462,0.017250,0.0,0.506997,0.040154,0.018376,0.0
391,Standing Up,0.488846,0.020633,-0.526126,0.999841,0.502742,-0.008212,-0.504925,0.999583,0.512030,...,0.000906,0.0,0.508203,0.033624,0.010968,0.0,0.509305,0.031508,0.011511,0.0
392,Standing Up,0.536482,0.025484,-0.605942,0.999599,0.558236,-0.007861,-0.577477,0.998839,0.570211,...,0.006773,0.0,0.574597,0.015530,0.021598,0.0,0.577846,0.011801,0.022592,0.0


In [77]:
df[df['class']=='Standing Up']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
389,Standing Up,0.483750,0.047641,-0.791061,0.999933,0.508691,0.001452,-0.762163,0.999869,0.522984,...,0.000877,0.0,0.543293,-0.020725,0.017091,0.0,0.548040,-0.028908,0.018152,0.0
390,Standing Up,0.473050,0.031469,-0.445751,0.999833,0.487423,0.001360,-0.426568,0.999568,0.496246,...,0.001607,0.0,0.505612,0.043462,0.017250,0.0,0.506997,0.040154,0.018376,0.0
391,Standing Up,0.488846,0.020633,-0.526126,0.999841,0.502742,-0.008212,-0.504925,0.999583,0.512030,...,0.000906,0.0,0.508203,0.033624,0.010968,0.0,0.509305,0.031508,0.011511,0.0
392,Standing Up,0.536482,0.025484,-0.605942,0.999599,0.558236,-0.007861,-0.577477,0.998839,0.570211,...,0.006773,0.0,0.574597,0.015530,0.021598,0.0,0.577846,0.011801,0.022592,0.0


In [55]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [57]:
y_test

40     Thumbs Up
44     Thumbs Up
304       Drowsy
59     Thumbs Up
267       Drowsy
         ...    
162    Thumbs Up
385       Drowsy
253       Drowsy
133    Thumbs Up
262       Drowsy
Name: class, Length: 118, dtype: object

### 2 Train Machine Learning Classification Model

In [58]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler # for standardizing the data so that no one feture overshadows another

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [59]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [60]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

C:\Users\Pratik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [61]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [62]:
fit_models['rf'].predict(X_test)

array(['Thumbs Up', 'Thumbs Up', 'Drowsy', 'Thumbs Up', 'Drowsy',
       'Drowsy', 'Drowsy', 'Drowsy', 'Thumbs Up', 'Drowsy', 'Thumbs Up',
       'Drowsy', 'Thumbs Up', 'Drowsy', 'Drowsy', 'Thumbs Up',
       'Thumbs Up', 'Drowsy', 'Thumbs Up', 'Drowsy', 'Drowsy', 'Drowsy',
       'Drowsy', 'Drowsy', 'Thumbs Up', 'Thumbs Up', 'Drowsy', 'Drowsy',
       'Thumbs Up', 'Thumbs Up', 'Drowsy', 'Thumbs Up', 'Drowsy',
       'Thumbs Up', 'Thumbs Up', 'Drowsy', 'Drowsy', 'Thumbs Up',
       'Drowsy', 'Thumbs Up', 'Thumbs Up', 'Drowsy', 'Drowsy', 'Drowsy',
       'Thumbs Up', 'Drowsy', 'Thumbs Up', 'Drowsy', 'Thumbs Up',
       'Drowsy', 'Drowsy', 'Thumbs Up', 'Drowsy', 'Drowsy', 'Drowsy',
       'Drowsy', 'Thumbs Up', 'Thumbs Up', 'Thumbs Up', 'Drowsy',
       'Drowsy', 'Drowsy', 'Thumbs Up', 'Thumbs Up', 'Thumbs Up',
       'Drowsy', 'Drowsy', 'Drowsy', 'Thumbs Up', 'Drowsy', 'Drowsy',
       'Drowsy', 'Thumbs Up', 'Drowsy', 'Thumbs Up', 'Drowsy',
       'Thumbs Up', 'Drowsy', 'Drowsy', 'Drows

### 3 Evaluate and Serialize Model 

In [63]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [64]:
for algo, model in fit_models.items():
    pred = model.predict(X_test)
    print(algo, accuracy_score(y_test, pred))

lr 1.0
rc 1.0
rf 1.0
gb 1.0


In [65]:
fit_models['rf'].predict(X_test)

array(['Thumbs Up', 'Thumbs Up', 'Drowsy', 'Thumbs Up', 'Drowsy',
       'Drowsy', 'Drowsy', 'Drowsy', 'Thumbs Up', 'Drowsy', 'Thumbs Up',
       'Drowsy', 'Thumbs Up', 'Drowsy', 'Drowsy', 'Thumbs Up',
       'Thumbs Up', 'Drowsy', 'Thumbs Up', 'Drowsy', 'Drowsy', 'Drowsy',
       'Drowsy', 'Drowsy', 'Thumbs Up', 'Thumbs Up', 'Drowsy', 'Drowsy',
       'Thumbs Up', 'Thumbs Up', 'Drowsy', 'Thumbs Up', 'Drowsy',
       'Thumbs Up', 'Thumbs Up', 'Drowsy', 'Drowsy', 'Thumbs Up',
       'Drowsy', 'Thumbs Up', 'Thumbs Up', 'Drowsy', 'Drowsy', 'Drowsy',
       'Thumbs Up', 'Drowsy', 'Thumbs Up', 'Drowsy', 'Thumbs Up',
       'Drowsy', 'Drowsy', 'Thumbs Up', 'Drowsy', 'Drowsy', 'Drowsy',
       'Drowsy', 'Thumbs Up', 'Thumbs Up', 'Thumbs Up', 'Drowsy',
       'Drowsy', 'Drowsy', 'Thumbs Up', 'Thumbs Up', 'Thumbs Up',
       'Drowsy', 'Drowsy', 'Drowsy', 'Thumbs Up', 'Drowsy', 'Drowsy',
       'Drowsy', 'Thumbs Up', 'Drowsy', 'Thumbs Up', 'Drowsy',
       'Thumbs Up', 'Drowsy', 'Drowsy', 'Drows

In [66]:
y_test

40     Thumbs Up
44     Thumbs Up
304       Drowsy
59     Thumbs Up
267       Drowsy
         ...    
162    Thumbs Up
385       Drowsy
253       Drowsy
133    Thumbs Up
262       Drowsy
Name: class, Length: 118, dtype: object

In [67]:
# Exporting our best Model in a file and save it

with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

#  Make Detections with Model

In [68]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [69]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [ ]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
                
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #  Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        #  Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        #  Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        #  Body Pose 
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
        

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [71]:
tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))

(381, 157)